In [36]:
import wilds
from wilds.datasets.waterbirds_dataset import WaterbirdsDataset
from torch.utils.data import Subset
import numpy as np

In [59]:
def majority_only_waterbirds_dataset(dataset:WaterbirdsDataset) -> WaterbirdsDataset:
    train_split_mask = dataset.split_array == dataset.split_dict["train"]
    train_split_idx = np.where(train_split_mask)[0]

    train_metadata_array = dataset.metadata_array[train_split_mask,:]

    #(train_metadata_array[:,0].numpy() == train_metadata_array[:,1].numpy()).mean()
    spurious = (train_metadata_array[:,0].numpy() == train_metadata_array[:,1].numpy())

    non_spurious_train_split_idx = train_split_idx[~spurious]

    dataset.split_array[non_spurious_train_split_idx] = -1

    return dataset

In [60]:
majority_only = True
dataset = WaterbirdsDataset(root_dir="datasets")
if majority_only:
    dataset = majority_only_waterbirds_dataset(dataset)
data_train = dataset.get_subset("train")
data_test  = dataset.get_subset("test")
data_valid = dataset.get_subset("val") 


In [67]:
dataset.metadata_fields.index('background')

0

In [25]:
def check_stats(data):

    spurious = (data.metadata_array[:,0].numpy() == data.metadata_array[:,1].numpy())
    confounder_strength = spurious.mean()

    print(f"Confounder_strength = {confounder_strength:.3f}")
    
    return confounder_strength



In [37]:
def majority_only(data):

    spurious = (data.metadata_array[:,0].numpy() == data.metadata_array[:,1].numpy())

    spurious_idx= np.arange(len(data))[spurious]

    return data[spurious_idx]
